In [40]:
import json
import requests
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [46]:
# Data Source:https://www.kaggle.com/datasets/sarahquesnelle/canada-data
Temperature_DF = pd.read_csv("Resources/Canada_Temperature_Data.csv")

Temperature_DF2 = Temperature_DF.loc[(Temperature_DF["Year"] >= 1989) & (Temperature_DF["Year"] <= 2020), :]

Prov_DF = Temperature_DF2.loc[Temperature_DF2["Prov"].isin(["ON", "AB", "QC"]), :]

Canada_Temperature_DF = Prov_DF.drop("Stn_Name", axis=1)

Canada_Temperature_DF


# define the conditions for each season
conditions = [
    (Canada_Temperature_DF['Month'].isin([12, 1, 2])),
    (Canada_Temperature_DF['Month'].isin([3, 4, 5])),
    (Canada_Temperature_DF['Month'].isin([6, 7, 8])),
    (Canada_Temperature_DF['Month'].isin([9, 10, 11]))
]

# define the corresponding values for each season
values = ['Winter', 'Spring', 'Summer', 'Fall']

# create a new column 'Season' based on the conditions and values
Canada_Temperature_DF['Seasons'] = np.select(conditions, values)




Canada_Temperature_DF.rename(columns={'Prov': 'Province','Tm': 'Average Temperature', 'S': 'Average Snowfall Recorded', 'P': 'Average Precipitation Recorded'}, inplace=True)



Canada_Temperature_DF['Average Temperature'] = Canada_Temperature_DF['Average Temperature'].astype(float)
Canada_Temperature_DF['Average Snowfall Recorded'] = Canada_Temperature_DF['Average Snowfall Recorded'].astype(float)
Canada_Temperature_DF['Average Precipitation Recorded'] = Canada_Temperature_DF['Average Precipitation Recorded'].astype(float)


Canada_Temperature_DF

,Year,Month,Province,Average Temperature,Average Snowfall Recorded,Average Precipitation Recorded,Seasons
941030,1989,1,AB,-10.4,27.0,35.5,Winter
941031,1989,1,AB,-11.1,24.0,28.0,Winter
941032,1989,1,AB,-12.8,37.5,43.5,Winter
941033,1989,1,AB,-9.1,36.0,54.0,Winter
941034,1989,1,AB,-12.9,20.2,23.2,Winter
...,...,...,...,...,...,...,...
1357243,2017,9,QC,13.5,0.0,62.4,Fall
1357244,2017,9,QC,14.5,0.0,83.8,Fall
1357245,2017,9,QC,14.1,0.0,52.2,Fall
1357246,2017,9,QC,9.3,0.0,76.4,Fall


In [47]:
# group by year, season, and province
grouped = Canada_Temperature_DF.groupby(['Year', 'Seasons', 'Province'])

# calculate summary statistics
summary_stats = grouped.agg({'Average Temperature': ['mean', 'median', 'std', 'var', 'sem'],
                             'Average Snowfall Recorded': ['mean', 'median', 'std', 'var', 'sem'],
                             'Average Precipitation Recorded': ['mean', 'median', 'std', 'var', 'sem']})

# flatten multi-level column names
summary_stats.columns = ['_'.join(col).strip() for col in summary_stats.columns.values]

# reset index
summary_stats = summary_stats.reset_index()


Prov_ON = summary_stats.loc[summary_stats["Province"].isin(["ON"]), :]
Prov_AB = summary_stats.loc[summary_stats["Province"].isin(["AB"]), :]
Prov_QC = summary_stats.loc[summary_stats["Province"].isin(["QC"]), :]

# print summary statistics by year, season, and province
print(Prov_AB)



     Year Seasons Province  Average Temperature_mean  \
0    1989    Fall       AB                  4.543175   
3    1989  Spring       AB                  3.222601   
6    1989  Summer       AB                 15.048938   
9    1989  Winter       AB                -10.994872   
12   1990    Fall       AB                  4.170015   
..    ...     ...      ...                       ...   
333  2016  Winter       AB                 -7.712791   
336  2017    Fall       AB                  2.961905   
339  2017  Spring       AB                  3.525373   
342  2017  Summer       AB                 16.726984   
345  2017  Winter       AB                 -8.559091   

     Average Temperature_median  Average Temperature_std  \
0                          5.05                 5.929142   
3                          5.25                 7.314569   
6                         15.20                 2.449415   
9                        -11.10                 4.073731   
12                         

In [48]:
# save summary_stats to Excel file
summary_stats.to_excel('summary_stats.xlsx', index=False)

In [49]:
provices_result = pd.read_csv("Resources/Economic_Sector_Prov.csv")
provices_result.head()

,Year,Region,Index,Source,Category,Sub-category,Sub-sub-category,Total,CO2eq,Unit
0,1990,Alberta,0,Provincial Inventory Total,NaN,NaN,NaN,True,165.65,Mt
1,1990,Alberta,1,Oil and Gas,NaN,NaN,NaN,True,61.96,Mt
2,1990,Alberta,2,Oil and Gas,Upstream Oil and Gas,NaN,NaN,True,58.36,Mt
3,1990,Alberta,3,Oil and Gas,Upstream Oil and Gas,Natural Gas Production and Processing,NaN,False,23.94,Mt
4,1990,Alberta,4,Oil and Gas,Upstream Oil and Gas,Conventional Oil Production,NaN,True,15.49,Mt


In [50]:
Gas_DF = provices_result.loc[provices_result["Source"].isin(["Oil and Gas"]), :]
Gas_DF

,Year,Region,Index,Source,Category,Sub-category,Sub-sub-category,Total,CO2eq,Unit
1,1990,Alberta,1,Oil and Gas,NaN,NaN,NaN,True,61.96,Mt
2,1990,Alberta,2,Oil and Gas,Upstream Oil and Gas,NaN,NaN,True,58.36,Mt
3,1990,Alberta,3,Oil and Gas,Upstream Oil and Gas,Natural Gas Production and Processing,NaN,False,23.94,Mt
4,1990,Alberta,4,Oil and Gas,Upstream Oil and Gas,Conventional Oil Production,NaN,True,15.49,Mt
5,1990,Alberta,5,Oil and Gas,Upstream Oil and Gas,Conventional Oil Production,Conventional Light Oil Production,False,9.77,Mt
...,...,...,...,...,...,...,...,...,...,...
17075,2020,Saskatchewan,14,Oil and Gas,Downstream Oil and Gas,Petroleum Refining,NaN,False,1.19,Mt
17076,2020,Saskatchewan,15,Oil and Gas,Downstream Oil and Gas,Natural Gas Distribution,NaN,False,0.22,Mt
17111,2020,Yukon,1,Oil and Gas,NaN,NaN,NaN,True,0.00,Mt
17112,2020,Yukon,2,Oil and Gas,Upstream Oil and Gas,NaN,NaN,True,0.00,Mt


In [51]:
Gas_DF_total = Gas_DF.loc[Gas_DF['Category'].isna()]
Gas_DF_total

,Year,Region,Index,Source,Category,Sub-category,Sub-sub-category,Total,CO2eq,Unit
1,1990,Alberta,1,Oil and Gas,NaN,NaN,NaN,True,61.96,Mt
49,1990,British Columbia,1,Oil and Gas,NaN,NaN,NaN,True,7.87,Mt
92,1990,Canada,1,Oil and Gas,NaN,NaN,NaN,True,102.65,Mt
141,1990,Manitoba,1,Oil and Gas,NaN,NaN,NaN,True,1.54,Mt
183,1990,New Brunswick,1,Oil and Gas,NaN,NaN,NaN,True,1.22,Mt
...,...,...,...,...,...,...,...,...,...,...
16949,2020,Ontario,1,Oil and Gas,NaN,NaN,NaN,True,7.19,Mt
16991,2020,Prince Edward Island,1,Oil and Gas,NaN,NaN,NaN,True,0.00,Mt
17026,2020,Quebec,1,Oil and Gas,NaN,NaN,NaN,True,2.27,Mt
17065,2020,Saskatchewan,1,Oil and Gas,NaN,NaN,NaN,True,17.32,Mt


In [52]:
Gas_DF_total.reset_index(inplace=True)
Gas_DF_total.drop('Index', axis=1, inplace=True)

Gas_DF_total





C:\Users\fabio\AppData\Local\Temp\ipykernel_11100\3186528816.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Gas_DF_total.drop('Index', axis=1, inplace=True)


,index,Year,Region,Source,Category,Sub-category,Sub-sub-category,Total,CO2eq,Unit
0,1,1990,Alberta,Oil and Gas,NaN,NaN,NaN,True,61.96,Mt
1,49,1990,British Columbia,Oil and Gas,NaN,NaN,NaN,True,7.87,Mt
2,92,1990,Canada,Oil and Gas,NaN,NaN,NaN,True,102.65,Mt
3,141,1990,Manitoba,Oil and Gas,NaN,NaN,NaN,True,1.54,Mt
4,183,1990,New Brunswick,Oil and Gas,NaN,NaN,NaN,True,1.22,Mt
...,...,...,...,...,...,...,...,...,...,...
398,16949,2020,Ontario,Oil and Gas,NaN,NaN,NaN,True,7.19,Mt
399,16991,2020,Prince Edward Island,Oil and Gas,NaN,NaN,NaN,True,0.00,Mt
400,17026,2020,Quebec,Oil and Gas,NaN,NaN,NaN,True,2.27,Mt
401,17065,2020,Saskatchewan,Oil and Gas,NaN,NaN,NaN,True,17.32,Mt
